<a href="https://colab.research.google.com/github/kluo9/Deap-Learning/blob/main/Anomaly_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This notebook presents unsupervised anomaly detection in images.
The main purpose is to train a model to determine whether the given image is similar with the training data.

The training data includes 100000 human faces, and the testing data has about 10000 from the same distribution with training data (label 0), and about 10000 from another distribution (anomalies, label 1).

The method is to train an autoencoder with the training data. 
Ideally the autoencoder should have small reconstruction error in the training set. 
During inference, we can use reconstruction error as anomaly score. Anomaly score can be seen as the degree of abnormality of an image. An image from unseen distribution should have higher reconstruction error.

The evaluation is ROC AUC score.

In [1]:
# Download data
!wget https://github.com/MachineLearningHW/HW8_Dataset/releases/download/v1.0.0/data.zip
!unzip data.zip

--2023-03-26 20:39:07--  https://github.com/MachineLearningHW/HW8_Dataset/releases/download/v1.0.0/data.zip
Resolving github.com (github.com)... 140.82.113.3
Connecting to github.com (github.com)|140.82.113.3|:443... connected.
HTTP request sent, awaiting response... 404 Not Found
2023-03-26 20:39:07 ERROR 404: Not Found.

unzip:  cannot find or open data.zip, data.zip.zip or data.zip.ZIP.


In [3]:
# Training progress bar
!pip install -q qqdm

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 120.9/120.9 KB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.6/83.6 KB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 27.3 MB/s eta 0:00:00


In [4]:
import random
import numpy as np
import torch
from torch import nn
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler, TensorDataset
import torchvision.transforms as transforms
import torch.nn.functional as F
from torch.autograd import Variable
import torchvision.models as models
from torch.optim import Adam, AdamW
from qqdm import qqdm, format_str
import pandas as pd